## Imports

In [139]:
import numpy as np
import json
import random
from datetime import date, datetime

import pandas as pd
from pandas import json_normalize
from pandas.api.types import CategoricalDtype

import panel.widgets as pnw
import panel as pn

from bokeh.layouts import row,column, gridplot
from bokeh.models import (ColumnDataSource, DataTable, HoverTool, IntEditor,
                          NumberEditor, NumberFormatter, SelectEditor,
                          StringEditor, StringFormatter, TableColumn)
from bokeh.plotting import figure, show
from bokeh.transform import factor_cmap
from bokeh.transform import factor_cmap, factor_mark
from bokeh.palettes import RdBu
from bokeh.io import curdoc
pn.extension()

## Data Transform

In [140]:
json_file_path = '../data/sample.json'    
with open(json_file_path, 'r') as file:
    data = json.load(file)
df = json_normalize(data['returnValue'])
df['initialDispatch'] = pd.to_datetime(df['initialDispatch'])
df['date'] = df['initialDispatch'].dt.date
month_names = [
    'January', 'February', 'March', 'April', 'May', 'June',
    'July', 'August', 'September', 'October', 'November', 'December'
]
month_cat_type = CategoricalDtype(
    categories=month_names,
    ordered=True
)
df['month_name'] = df['initialDispatch'].dt.month_name()
df['month_name'] = df['month_name'].astype(month_cat_type)

units = ['E131', 'M122', 'E132', 'M133']
df['unit'] = [random.choice(units) for _ in range(len(df))]
df['unit'] = df['unit'].astype(str)
avg_response_times = df.groupby('unit')['initialResponseMinutes'].mean().reset_index()
print(df.head(1))

         callId               initialDispatch natureCode  \
0  CFD-23000001 2023-06-16 05:26:55.381662700     AFACom   

           streetAddress   longitude  latitude  initialResponseMinutes  \
0  3852 NE Washington Av -123.338486  44.58631                9.978979   

   fullComplementMinutes                                              notes  \
0               9.978979  CFD-23000001 AFACom 6/16/2023 5:26 AM 3852 NE ...   

         date month_name  unit  
0  2023-06-16       June  E132  


## Creating Chart Functions

In [7]:
def create_bar_chart(dataframe):
    bar_chart = figure(title='Average Initial Response Time by Unit', x_axis_label='Unit', y_axis_label='Avg Initial Response Time (minutes)', x_range=dataframe['unit'])
    bar_chart.vbar(x='unit', top='initialResponseMinutes', source=dataframe, width=0.7, color="navy")
    return bar_chart

    
# Function to create a histogram using Bokeh
def create_histogram(dataframe):
    # Calculate histogram data
    hist, edges = np.histogram(dataframe['initialResponseMinutes'], bins=20)
    hist_df = pd.DataFrame({'hist': hist, 'left': edges[:-1], 'right': edges[1:]})
    source = ColumnDataSource(hist_df)
    
    # Create the histogram figure
    hist_fig = figure(title='Histogram', x_axis_label='Initial Response Minutes', y_axis_label='Count')
    hist_fig.quad(bottom=0, top='hist', left='left', right='right', source=source, fill_color="navy", line_color="white", alpha=0.5)
    
    return hist_fig

## Chart Previews

In [8]:
import pandas as pd
import panel as pn
from bokeh.models import DatetimeRangeSlider
from datetime import datetime

# Callback function for the range slider
def update_plot(attr, old, new):
    start, end = new  # Unpack the tuple to get start and end
    start = pd.to_datetime(start, unit='ms')  # Convert from Unix time in milliseconds
    end = pd.to_datetime(end, unit='ms')  # Convert from Unix time in milliseconds

    # Filter the data based on the selected date range
    filtered = df[(df['initialDispatch'] >= start) & (df['initialDispatch'] <= end)]
    
    # Update the data in the plot (You'll need to define create_bar_chart function)
    new_avg_response_times = filtered.groupby('unit')['initialResponseMinutes'].mean().reset_index()
    source.data = new_avg_response_times
    

min_date_timestamp = df['initialDispatch'].min().timestamp() * 1000  # Convert to milliseconds
max_date_timestamp = df['initialDispatch'].max().timestamp() * 1000  # Convert to milliseconds

# Create the range slider with timestamps
date_range_slider = DatetimeRangeSlider(start=min_date_timestamp, end=max_date_timestamp, 
                                        value=(min_date_timestamp, max_date_timestamp), 
                                        step=1, title="Date Range")

date_range_slider.on_change('value', update_plot)

# Create the plots
bar_chart = create_bar_chart(avg_response_times)
histogram = create_histogram(df)

# Convert the plots to Panel panes
line_chart_pane = pn.pane.Bokeh(bar_chart)
histogram_pane = pn.pane.Bokeh(histogram)

# Create a row layout for the line chart and histogram
chart_row = pn.Row(line_chart_pane, histogram_pane)

# Arrange plots and widgets in a layout
layout = pn.Column(date_range_slider, chart_row)

# Serve the app
layout.servable()


Column
    [0] Bokeh(DatetimeRangeSlider)
    [1] Row
        [0] Bokeh(figure)
        [1] Bokeh(figure)

## Working Daterange Slider

In [132]:
import pandas as pd
import panel as pn
from bokeh.models import ColumnDataSource, DatetimeRangeSlider
from datetime import datetime

# Create a sample DataFrame
data = {
    'Date': pd.date_range(start='2023-01-01', periods=90, freq='D'),
    'Value': range(90)
}
df = pd.DataFrame(data)

# Create a Panel app
def update_plot(attr, old, new):
    start, end = new  # Unpack the tuple to get start and end
    
    start_date = datetime.fromtimestamp(start / 1000)  # Convert to seconds
    end_date = datetime.fromtimestamp(end / 1000)      # Convert to seconds
    
    filtered_df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]
    source.data = ColumnDataSource.from_df(filtered_df)

# Create the initial plot
source = ColumnDataSource(df)
p = figure(x_axis_label='Date', y_axis_label='Value', x_axis_type='datetime')
p.line(x='Date', y='Value', source=source)

# Create the range slider
min_date_timestamp = df['Date'].min().timestamp() * 1000
max_date_timestamp = df['Date'].max().timestamp() * 1000
date_range_slider = DatetimeRangeSlider(start=min_date_timestamp, end=max_date_timestamp, 
                                        value=(min_date_timestamp, max_date_timestamp), 
                                        step=1, title="Date Range")

# Attach the callback to the range slider
date_range_slider.on_change('value', update_plot)

layout = pn.Column(date_range_slider, p)
layout.servable()


Column
    [0] Bokeh(DatetimeRangeSlider)
    [1] Bokeh(figure)

In [141]:
#output_notebook()
def wgs84_to_web_mercator(df, lon="longitude", lat="latitude"):
    k = 6378137
    df["x"] = df[lon] * (k * np.pi/180.0)
    df["y"] = np.log(np.tan((90 + df[lat]) * np.pi/360.0)) * k
    return df

df = wgs84_to_web_mercator(df)
expected_factors = ['E132', 'M122']
color_mapper = factor_cmap(field_name='unit', palette=[RdBu[3][1], RdBu[3][0]], factors=expected_factors)
df['color'] = [RdBu[3][1] if 'E' in unit else RdBu[3][0] for unit in df['unit']]
source = ColumnDataSource(df)


columns = [
    TableColumn(field="callId", title="Call ID", formatter=StringFormatter()),
    TableColumn(field="initialDispatch", title="Initial Dispatch", formatter=StringFormatter()),
    TableColumn(field="natureCode", title="Nature Code", formatter=StringFormatter()),
    TableColumn(field="streetAddress", title="Street Address", formatter=StringFormatter()),
    TableColumn(field="longitude", title="Longitude (Web Mercator)", formatter=StringFormatter()),
    TableColumn(field="latitude", title="Latitude (Web Mercator)", formatter=StringFormatter()),
    TableColumn(field="initialResponseMinutes", title="Initial Response Minutes", formatter=StringFormatter()),
    TableColumn(field="fullComplementMinutes", title="Full Complement Minutes", formatter=StringFormatter()),
    TableColumn(field="notes", title="Notes", formatter=StringFormatter()),
    TableColumn(field="date", title="Date", formatter=StringFormatter()),
    TableColumn(field="month_name", title="Month Name", formatter=StringFormatter()),
    TableColumn(field="unit", title="Unit", formatter=StringFormatter()),
    TableColumn(field="x", title="X (Web Mercator)", formatter=StringFormatter()),
    TableColumn(field="y", title="Y (Web Mercator)", formatter=StringFormatter()),
]

data_table = DataTable(source=source, columns=columns, editable=True, width=800, index_position=-1, index_header="row index", index_width=60)

#Coordinate conversion to Mercator, required for Bokeh
x_min, x_max = df['x'].min() - 1000, df['x'].max() + 1000
y_min, y_max = df['y'].min() - 1000, df['y'].max() + 1000
p = figure(x_range=(x_min, x_max), y_range=(y_min, y_max),
           x_axis_type="mercator", y_axis_type="mercator",
           width=800, height=600, tools="pan,wheel_zoom,lasso_select,reset", active_drag="lasso_select")

p.add_tile(OSM)
p.circle(x='x', y='y', size=10, fill_color='color', fill_alpha=0.8, source=source, legend_field='unit')

map_pane = pn.pane.Bokeh(p)
table_pane = pn.pane.Bokeh(data_table)
layout = pn.Column(map_pane, table_pane)
layout.servable()


Column
    [0] Bokeh(figure)
    [1] Bokeh(DataTable)

In [6]:
#output_notebook()
def wgs84_to_web_mercator(df, lon="longitude", lat="latitude"):
    k = 6378137
    df["x"] = df[lon] * (k * np.pi/180.0)
    df["y"] = np.log(np.tan((90 + df[lat]) * np.pi/360.0)) * k
    return df
    
df = wgs84_to_web_mercator(df)
expected_factors = ['E132', 'M122']
color_mapper = factor_cmap(field_name='unit', palette=[RdBu[3][1], RdBu[3][0]], factors=expected_factors)
df['color'] = [RdBu[3][1] if 'E' in unit else RdBu[3][0] for unit in df['unit']]
source = ColumnDataSource(df)


columns = [
    TableColumn(field="callId", title="Call ID", formatter=StringFormatter()),
    TableColumn(field="initialDispatch", title="Initial Dispatch", formatter=StringFormatter()),
    TableColumn(field="natureCode", title="Nature Code", formatter=StringFormatter()),
    TableColumn(field="streetAddress", title="Street Address", formatter=StringFormatter()),
    TableColumn(field="longitude", title="Longitude (Web Mercator)", formatter=StringFormatter()),
    TableColumn(field="latitude", title="Latitude (Web Mercator)", formatter=StringFormatter()),
    TableColumn(field="initialResponseMinutes", title="Initial Response Minutes", formatter=StringFormatter()),
    TableColumn(field="fullComplementMinutes", title="Full Complement Minutes", formatter=StringFormatter()),
    TableColumn(field="notes", title="Notes", formatter=StringFormatter()),
    TableColumn(field="date", title="Date", formatter=StringFormatter()),
    TableColumn(field="month_name", title="Month Name", formatter=StringFormatter()),
    TableColumn(field="unit", title="Unit", formatter=StringFormatter()),
    TableColumn(field="x", title="X (Web Mercator)", formatter=StringFormatter()),
    TableColumn(field="y", title="Y (Web Mercator)", formatter=StringFormatter()),
]

data_table = DataTable(source=source, columns=columns, editable=True, width=800, index_position=-1, index_header="row index", index_width=60)

#Coordinate conversion to Mercator, required for Bokeh
x_min, x_max = df['x'].min() - 1000, df['x'].max() + 1000
y_min, y_max = df['y'].min() - 1000, df['y'].max() + 1000
p = figure(x_range=(x_min, x_max), y_range=(y_min, y_max),
           x_axis_type="mercator", y_axis_type="mercator",
           width=800, height=600, tools="pan,wheel_zoom,lasso_select,reset", active_drag="lasso_select")

p.add_tile(OSM)
p.circle(x='x', y='y', size=10, fill_color='color', fill_alpha=0.8, source=source, legend_field='unit')

# Callback function to update table based on lasso selection
def update_table(attr, old, new):
    selected_indices = source.selected.indices
    if selected_indices:
        filtered_data = df.iloc[selected_indices]
    else:
        filtered_data = df

    table_source.data = ColumnDataSource.from_df(filtered_data)

# Set up separate data source for the table
table_source = ColumnDataSource(df)
source.selected.on_change('indices', update_table)
data_table = DataTable(source=table_source, columns=columns, editable=True, width=800)

# Panel layout
map_pane = pn.pane.Bokeh(p)
table_pane = pn.pane.Bokeh(data_table)
layout = pn.Column(map_pane, table_pane)
layout.servable()


Column
    [0] Bokeh(figure)
    [1] Bokeh(DataTable)